## TSP (Traveling Salesman Problem)



In [1]:
import math
import random
import turtle as t


In [2]:
def createCities(n):
    cities = [(random.randint(0,1800),random.randint(0,1200)) for i in range(n)]
    return cities

def dist1(m,n):
    x1,y1 = cities[m][0], cities[m][1]
    x2,y2 = cities[n][0], cities[n][1]
    return round(math.sqrt((x1-x2)**2 + (y1-y2)**2))

def dist(tour):
    summe = 0
    for i in range(len(cities)-1):
         summe += dist1(tour[i],tour[i+1])
    return summe + dist1(tour[-1],tour[0])

def showCities(label = False):
    t.tracer(False)
    t.hideturtle()
    for i in range(len(cities)):
        x, y  = cities[i][0], cities[i][1]
        t.penup()
        t.setposition(x,y)
        t.pendown()
        if i == 0:
            t.dot(13,'red')
        else:
            t.dot(8,'black')
        if label: t.write(i)
        t.update()

def showTour(tour):
    t.hideturtle()
    t.tracer(True)
    t.speed(2)
    t.penup()
    x0 = cities[tour[0]][0]
    y0 = cities[tour[0]][1]
    t.setposition(x0,y0)
    t.pendown()
    for i in range(1,len(tour)):
        x = cities[tour[i]][0]
        y = cities[tour[i]][1]
        t.goto(x,y)
    t.goto(x0,y0)
    t.up()
    t.goto(50,50)
    t.down()
    t.write(round(dist(tour)))
   

### Brute Force

In [3]:
import itertools
def brute():
    mindist = float('inf')
    mintour = []
    touren = itertools.permutations(range(len(cities)))
    for tour in touren:
        if dist(tour) < mindist:
            mindist = dist(tour)
            mintour = tour
    return mintour

In [ ]:
t.setworldcoordinates(0, 0, 1800, 1200)
random.seed(37)
cities = createCities(9)
showCities()
tour = brute()
showTour(tour)
t.mainloop()

### Greedy

In [3]:
def berlin52():
    return [[565,575],[25,185],[345,750],[945,685],[845,655],
    [880,660],[25,230],[525,1000],[580,1175],[650,1130],[1605,620],
    [1220,580],[1465,200],[1530,5],[845,680],[725,370],[145,665],
    [415,635],[510,875],[560,365],[300,465],[520,585],[480,415],
    [835,625],[975,580],[1215,245],[1320,315],[1250,400],[660,180],
    [410,250],[420,555],[575,665],[1150,1160],[700,580],[685,595],
    [685,610],[770,610],[795,645],[720,635],[760,650],[475,960],
    [95,260],[875,920],[700,500],[555,815],[830,485],[1170,65],
    [830,610],[605,625],[595,360],[1340,725],[1740,245]]

def nearest(cities,i,iset):
    '''
    i : index in cities
    cities: list von city-koordinaten
    iset: set von indizes, aus denen die nächstgelegene Stadt zu cities[i]
        gesucht wird
    return: index der nächstgelegenen Stadt
    '''
    bestIndex = -1
    bestDist = float('inf')
     
    for k in iset:
        if dist1(k,i) < bestDist:
            bestDist = dist1(k,i)
            bestIndex = k

    return bestIndex

def greedy(start=0):
    iset = set(range(len(cities)))
    k = start
    tour = []
    
    while iset:
        tour.append(k)
        iset.remove(k)
        k = nearest(cities,k,iset)
    return tour

In [4]:
t.setworldcoordinates(0, 0, 1800, 1200)
cities = berlin52()
showCities()
tour = greedy()
showTour(tour)
t.mainloop()

### Greedy + 2opt

In [5]:
def swap(tour,i,j):
    w1 = tour[:i] 
    w2 = [tour[i],tour[j]]
    w3 = tour[j-1:i:-1]  
    w4 = tour[j+1:] 
    return w1 + w2 + w3 + w4 

def twoopt1(tour):
    bestDist = dist(tour)
    besttour = tour[:]
    for i in range(len(tour)-1):
        for j in range(i+1,len(tour)):
                tmp = swap(tour,i,j)
                if dist(tmp) < bestDist:
                    besttour = tmp[:] 
                    bestDist = dist(tmp)
    return besttour

def twoopt(tour):
    tour1 = tour[:]
    tour2 = twoopt1(tour1)
    while tour2 != tour1:
        tour1 = tour2[:]
        tour2 = twoopt1(tour1)
    return tour1

In [7]:
t.setworldcoordinates(0, 0, 1800, 1200)
cities = berlin52()
showCities()
tour = greedy(44)  # 44
tour = twoopt(tour)
showTour(tour)
t.mainloop()

### Simulated Annealing

In [11]:
def neighbor(tour):
    i,j = sorted(random.sample(range(len(tour)),2))
    return swap(tour,i,j)

def anneal(tour):
    best_dist = dist(tour)
    best_tour = tour[:]
    T = 100
    T_min = 0.0005
    alpha = 0.995
    H = 125
    while T > T_min:
        for i in range(H):
            new_tour = neighbor(best_tour)
            new_dist = dist(new_tour)
            if (new_dist < best_dist):
                best_tour = new_tour[:]
                best_dist = new_dist
            elif (new_dist > best_dist):
                if random.random() <=  prob(new_dist, best_dist, T):
                    #print(T,new_dist,best_dist,prob(new_dist, best_dist, T))
                    best_tour = new_tour[:]
                    best_dist = new_dist
 
        T = T*alpha
    return best_tour 

def prob(new_dist,best_dist,T):
    return  math.exp(-(new_dist-best_dist)/T)

In [ ]:
random.seed(40)   # 40
t.setworldcoordinates(0, 0, 1800, 1200)
cities = berlin52()
showCities()
tour = greedy()
tour = anneal(tour)
showTour(tour)
t.mainloop()

### MST + 2opt

In [4]:
def fullgraph():
    G = {}
    for c in range(len(cities)):
        vmap = {}
        for d in range(len(cities)):
            if c != d:
                vmap[d] = dist1(c,d)
        G[c] = vmap
    return G

def showMst(mst):
    for v in mst:
        if mst[v] is not None:
            t.penup()
            x0, y0 = cities[v][0], cities[v][1]
            x1, y1 = cities[mst[v]][0], cities[mst[v]][1]
            t.setposition(x0,y0)
            t.pendown()
            t.setposition(x1,y1)
    t.update()
    
from heapq import heappop, heappush    
def jarnik_prim(G,start):  
    
    mst = {}                                 # leere Teillösung
    heap = [(0, None, start)]                # Heap mit Kanten und Kosten
    summe = 0
    while heap:
        cost, prev, u = heappop(heap)
        if u in mst: continue            # Zielknoten schon im Baum?
        mst[u] = prev                    # Kante von u nach prev kommt in mst
        summe += cost
        for v, cost in G[u].items():
            heappush(heap, (cost, u, v))
    return mst

In [5]:
t.setworldcoordinates(0, 0, 1800, 1200)
cities = berlin52()
showCities()
G = fullgraph()
mst = jarnik_prim(G,0)
showMst(mst)
t.mainloop()

In [6]:
def createMstGraph(mst):
    mstG = {v : set() for v in mst}
    for v in mst:
        if mst[v] is not None:
            mstG[v].add(mst[v])
            mstG[mst[v]].add(v)
    return mstG

def explore(v):
    global counter
    visited[v] = True
    previsit[v] = counter
    counter += 1
    for w in sorted(G[v]):  
        if not visited[w]:
            explore(w)

In [15]:
t.setworldcoordinates(0, 0, 1800, 1200)
cities = berlin52()
showCities(label=True)
G = fullgraph()
mst = jarnik_prim(G,0)
G = createMstGraph(mst)

counter = 0
visited =  {v : False for v in G}
previsit = {v : 0 for v in G}
explore(15)                         # 15 ist am besten
tour = sorted(G.keys(),key=lambda v: previsit[v])
tour = twoopt(tour)
showTour(tour)
t.mainloop()